In [2]:
import pandas as pd
df_ml = pd.read_csv("../data/processed/ML_Chile_2022-12-27.csv")
df_f = pd.read_csv("../data/processed/FalabellaAyuda_2022-12-27.csv")
df_r = pd.read_csv("../data/processed/RipleyChile_2022-12-27.csv")
df_p = pd.read_csv("../data/processed/tiendas_paris_2022-12-27.csv")
df = {
    "ML": df_ml,
    "Falabella": df_f,
    "RipleyChile": df_r,
    "Paris": df_p,
}

In [3]:
# eliminar nan
for key, value in df.items():
    print(key)
    value.dropna(subset=["tweet_text"], inplace=True)

ML
Falabella
RipleyChile
Paris


In [4]:
# medición de la longitud de los microtextos.
import statistics as stats
resultado = {
    "tienda": [],
    "count": [],
    "mean": [],
    "std": [],
    "multimode": [],
    "max": [],
    "min": [],
    "quantiles": [],
}
text_length = []
for key, value in df.items():
    for row in value.iterrows():
        text_length.append(row[1][3].__len__())
    resultado["tienda"].append(key)
    resultado["count"].append(text_length.__len__())
    resultado["mean"].append(stats.mean(text_length))
    resultado["std"].append(stats.stdev(text_length))
    resultado["multimode"].append(stats.multimode(text_length))
    resultado["max"].append(max(text_length))
    resultado["min"].append(min(text_length))
    resultado["quantiles"].append(stats.quantiles(text_length))
    text_length = []
df_resultado = pd.DataFrame(resultado)
df_resultado

,tienda,count,mean,std,multimode,max,min,quantiles
0,ML,77,143.701299,76.886120,"[122, 130, 236, 139, 113, 114, 152, 73]",280,12,"[84.0, 139.0, 223.0]"
1,Falabella,434,139.421659,75.698042,[113],275,1,"[79.0, 126.5, 208.25]"
2,RipleyChile,403,127.017370,75.470413,[111],278,5,"[67.0, 116.0, 185.0]"
3,Paris,362,129.825967,72.932444,"[111, 137, 81, 222]",279,2,"[72.5, 123.5, 188.0]"


In [17]:
import nltk
nltk.download("stopwords")
from collections import Counter

def word_frequency(data):
    addi = ['si', 'mas', 'aun']
    # Descarga la lista de stop words del español
    words0 = []
    stop_words = nltk.corpus.stopwords.words("spanish")
    for i in addi:
        stop_words.append(i)
    # Inicializa un contador de palabras en cero
    word_count = Counter()
    
    # Itera a través de cada cadena en la lista de datos
    for text in data:
        # Separa la cadena en una lista de palabras
        words = text.split()
        
        # Elimina las stop words de la lista de palabras
        words = [word for word in words if word not in stop_words]
        for word in words:
            words0.append(word)
        
        # Actualiza el contador de palabras con la lista de palabras filtrada
    word_count.update(words0)
    return word_count

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/unknownsystem/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
from autocorrect import Speller
spell = Speller('es')
texts = []
resultado = {}
for key, value in df.items():
    for row in value.iterrows():
        texts.append(spell(row[1][3]))
    resultado[key] = word_frequency(texts)

In [27]:
keys = {}
for key, value in resultado.items():
    keys[key] = {}
    for top in sorted(value, key=value.get, reverse=True)[:5]:
        keys[key][top] = value[top]
keys

{'ML': {'necesito': 21, 'compra': 16, 'cuenta': 15, 'hoy': 14, 'ahora': 12},
 'Falabella': {'compra': 86,
  'necesito': 74,
  'regalo': 63,
  'producto': 62,
  'pedido': 54},
 'RipleyChile': {'compra': 166,
  'necesito': 156,
  'producto': 114,
  'respuesta': 97,
  'regalo': 95},
 'Paris': {'compra': 231,
  'necesito': 220,
  'producto': 157,
  'respuesta': 133,
  'compre': 128}}